In [1]:
from lxml import etree as ET
import shutil
from subprocess import check_output
from pathlib import Path
import numpy as np
import os
#import matplotlib.pyplot as plt
import pandas as pd
#from shapely.geometry import Polygon,
import RClib as RClib



In [2]:
%load_ext autoreload
%autoreload 2
''
config = RClib.loadconfigs('.\config.json')
scandf = RClib.provide_scandf(inputdirectory=config['workspace'], imageformat = config['baseimageformat'])
previousscandf_path = Path(config['workspace']) / 'rcprocessingdf.pkl'
if previousscandf_path.is_file() and config['resume_processing']:
    previousscandf = pd.read_pickle(str(Path(config['workspace']) / 'rcprocessingdf.pkl'))
    previousscandf = previousscandf.set_index('id',drop=False)
    scandf = scandf.set_index('id',drop=False)
    print(previousscandf.columns)
    scandf  = pd.concat([scandf , previousscandf[previousscandf.columns.difference(scandf.columns)]], join='outer', axis=1)
    scandf.update(previousscandf)

print(scandf.columns)

scandf= scandf.apply(RClib.defineRawTherapeeOutput, foldername='02_PNG', axis=1)
scandf= scandf.apply(RClib.defineRealityCaptureOutput, foldername='03_RC', axis=1)
#rcsettings_template = pd.read_csv('.\\allRCsettings_template.csv', sep=';', header=0) 
#print(rcsettings_template)



Index(['id', 'processingstate', 'scan_dir', 'pp3file', 'gcpsfile', 'imagedf'], dtype='object')


In [3]:

scandf2=pd.DataFrame()

for index,scan in scandf.iterrows():
    if not scan['imagedf'] is None:
        if 'pp3file' in scan.keys() and scan['pp3file']:    
            scan['imagedf'] = scan['imagedf'].apply(RClib.developwithRawTherapee, pp3filepath=scan['pp3file'][0],  outputfolderpath=scan['RToutputfolder'], axis=1) 
        scan = RClib.makeImagelist(scan, imagelistname=scan['id'])
        scan = RClib.createRCproject(scan)
        
        if not 'commandlist_alignsettings' in scan['processingstate']:
            scan['commandlist_alignsettings']=Path(config['alignmentsettings'])
            RClib.executeRCCMDuseRCproject(scan, rccmdpathfield='commandlist_alignsettings' )
        if not 'align-rccmdpath' in scan['processingstate']:
            scan['commandlist_align'] = []
            scan['commandlist_align'].append('-add ' + str(scan['list_dev-img_path']))
            
            if config['use_gcps']:
                scan['commandlist_align'].append('-detectMarkers')
                if Path(config['constantgcps']).is_file() and Path(config['gcpsparamfile']).is_file():
                    scan['commandlist_align'].append('-importGroundControlPoints ' + str(config['constantgcps']) + ' ' + config['gcpsparamfile'])
                if not Path(config['constantgcps']).is_file() and Path(config['gcpsparamfile']).is_file() and scan['gcpsfile']:
                    scan['commandlist_align'].append('-importGroundControlPoints ' + str(scan['gcpsfile'][0]) + ' ' + config['gcpsparamfile'] )
            scan['commandlist_align'].append('-align')
            scan['commandlist_align'].append('-setReconstructionRegionAuto')
            scan['commandlist_align'].append('-exportReport ' + str(Path(scan['RCoutputfolder'] / Path(scan['id']+'.txt '))) + config['alignmentreport_template'])
            scan['commandlist_align'].append('-save')
            scan['commandlist_align'].append('-quit')
            scan = RClib.makeRCCMDfromListfield(scan, commandlistfield='commandlist_align', rccmdpathfield='align-rccmdpath')
            RClib.executeRCCMDuseRCproject(scan, rccmdpathfield='align-rccmdpath' )
        #scan['processingstatus'].append({'alignment_report': Path(scan['RCoutputfolder'] / Path(scan['id']+'.html '))})
        scandf2 = scandf2.append(scan)
#pd.write_csv(Path(config['workspace']) / 'rcprocessingdf.csv', sep=';')
scandf2.to_pickle(str(Path(config['workspace']) / 'rcprocessingdf.pkl'))

E:\01_Digitization\VAM\10_Room\VA_00959\03_RC\commandlist_align.rccmd


C:\Users\Jens\AppData\Local\Temp\ipykernel_21636\3796596262.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  scandf2 = scandf2.append(scan)


E:\01_Digitization\VAM\10_Room\VA_Ass_2259\03_RC\commandlist_align.rccmd


C:\Users\Jens\AppData\Local\Temp\ipykernel_21636\3796596262.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  scandf2 = scandf2.append(scan)


In [4]:
scandf3 = pd.read_pickle(str(Path(config['workspace']) / 'rcprocessingdf.pkl'))
#for index,scan in scandf3.iterrows():
        #if not 'reconstruct-rccmdpath' in scan['processingstate']:
            #scan['commandlist_reconstruct'] = []
            #scan['commandlist_reconstruct'].append('-calculateNormalModel')  
            #scan['commandlist_reconstruct'].append('-save')
            #scan['commandlist_reconstruct'].append('-quit')  
            #scan = RClib.makeRCCMDfromListfield(scan, commandlistfield='commandlist_reconstruct', rccmdpathfield='reconstruct-rccmdpath')
            #RClib.executeRCCMDuseRCproject(scan, rccmdpathfield='reconstruct-rccmdpath' )

E:\01_Digitization\VAM\10_Room\VA_00959\03_RC\commandlist_reconstruct.rccmd
